# <center>UrbanSound8K - Features Sensitivity to Processing Parameters</center>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Libraries-Import" data-toc-modified-id="Libraries-Import-1">Libraries Import</a></span></li><li><span><a href="#Paths" data-toc-modified-id="Paths-2">Paths</a></span></li><li><span><a href="#Import-of-the-Metadata-File" data-toc-modified-id="Import-of-the-Metadata-File-3">Import of the Metadata File</a></span></li><li><span><a href="#Choice-of-an-Event" data-toc-modified-id="Choice-of-an-Event-4">Choice of an Event</a></span></li><li><span><a href="#Audio-Processing-Parameters" data-toc-modified-id="Audio-Processing-Parameters-5">Audio Processing Parameters</a></span></li><li><span><a href="#Audio-Pre-Processing" data-toc-modified-id="Audio-Pre-Processing-6">Audio Pre-Processing</a></span></li><li><span><a href="#Spectrogram" data-toc-modified-id="Spectrogram-7">Spectrogram</a></span><ul class="toc-item"><li><span><a href="#FFT-Size" data-toc-modified-id="FFT-Size-7.1">FFT Size</a></span></li><li><span><a href="#Hop-Length" data-toc-modified-id="Hop-Length-7.2">Hop Length</a></span></li><li><span><a href="#Windowing-Function" data-toc-modified-id="Windowing-Function-7.3">Windowing Function</a></span></li></ul></li><li><span><a href="#Mel-Spectrogram" data-toc-modified-id="Mel-Spectrogram-8">Mel-Spectrogram</a></span><ul class="toc-item"><li><span><a href="#Number-of-Mel-Filterbanks" data-toc-modified-id="Number-of-Mel-Filterbanks-8.1">Number of Mel Filterbanks</a></span></li></ul></li><li><span><a href="#MFCC" data-toc-modified-id="MFCC-9">MFCC</a></span><ul class="toc-item"><li><span><a href="#Number-of-Coefficients" data-toc-modified-id="Number-of-Coefficients-9.1">Number of Coefficients</a></span></li></ul></li></ul></div>

## Libraries Import

In [ ]:
import os
import math
import collections
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
import IPython
import torch
import torch.nn as nn
import torchaudio
import torchaudio.transforms as transforms

## Paths

In [ ]:
dataset_path = "../dataset"
figures_path = "../figures"

## Import of the Metadata File

In [ ]:
metadata = pd.read_csv(os.path.join(dataset_path, "UrbanSound8K.csv"))
metadata

## Choice of an Event

For a recall, the different classes present in the UrbanSound8K dataset are:
- air_conditioner
- car_horn
- children_playing
- dog_bark
- drilling
- engine_idling
- gun_shot
- jackhammer
- siren
- street_music

In [ ]:
event = metadata[
    (metadata["class"] == "siren") & 
    (metadata["salience"] == 1) &
    ((metadata["end"] - metadata["start"]) == 4)
].sample(1)
display(event)

event_name = event["slice_file_name"].item()
event_fold = event["fold"].item()
event_class = event["class"].item()
event_path = os.path.join(dataset_path, f"fold{event_fold}", event_name)

display(IPython.display.Audio(filename=event_path))

## Audio Processing Parameters

In [ ]:
target_length = 4
target_sample_rate = 22050
n_samples = target_length * target_sample_rate

## Audio Pre-Processing

In [ ]:
# Load the audio signal
signal, sr = torchaudio.load(event_path)
# Mix it down to mono if necessary
if signal.shape[0] > 1:
    signal = torch.mean(signal, dim=0, keepdim=True)
# Resample it
resample_transform = transforms.Resample(sr, target_sample_rate)
signal = resample_transform(signal)
# Cut if necessary
if signal.shape[1] > n_samples:
    signal = signal[:, :n_samples]
# Right pad if necessary
if signal.shape[1] < n_samples:
    n_missing_samples = n_samples - signal.shape[1]
    last_dim_padding = (0, n_missing_samples)
    signal = nn.functional.pad(signal, last_dim_padding)

## Spectrogram

### FFT Size

In [ ]:
amplitude_to_db_transform = torchaudio.transforms.AmplitudeToDB(
                    # Scale of input tensor ("power" or "magnitude"). The power being the elementwise square of the magnitude. (Default: "power")
                    stype = "power",
                    # Minimum negative cut-off in decibels. A reasonable number is 80. (Default: None)
                    top_db = None
                    )

n_fft_values = [128, 256, 512, 1024]

n_cols = 2
n_rows = math.ceil(len(n_fft_values)/n_cols)

fig = plt.figure(figsize=(10,10))

for index, n_fft_value in enumerate(n_fft_values):
    
        spectrogram_transform = torchaudio.transforms.Spectrogram( 
                    # Size of FFT, creates n_fft // 2 + 1 bins. (Default: 400)
                    n_fft = n_fft_value,
                    # Window size. (Default: n_fft)
                    win_length = n_fft_value,
                    # Length of hop between STFT windows. (Default: win_length // 2)
                    hop_length = n_fft_value // 2,
                    # Two sided padding of signal. (Default: 0)
                    pad = 0,
                    # A function to create a window tensor that is applied/multiplied to each frame/window. (Default: torch.hann_window)
                    window_fn = torch.hann_window,
                    # Exponent for the magnitude spectrogram, (must be > 0) e.g., 1 for energy, 2 for power, etc. (Default: 2)
                    power = 2,
                    # Whether to normalize by magnitude after stft. (Default: False)
                    normalized = True,
                    # Arguments for window function. (Default: None)
                    wkwargs = None,
                    # Whether to pad waveform on both sides so that the t-th frame is centered at time t x hop_length (Default: True)
                    center = False,
                    # Controls the padding method used when center is True. (Default: "reflect")
                    pad_mode = "reflect",
                    # Controls whether to return half of results to avoid redundancy. (Default: True)
                    onesided = True,
                    # Indicates whether the resulting complex-valued Tensor should be represented with native complex dtype, 
                    # such as torch.cfloat and torch.cdouble, or real dtype mimicking complex value with an extra dimension 
                    # for real and imaginary parts. (See also torch.view_as_real.)
                    # This argument is only effective when power=None. It is ignored for cases where power is a number as in those cases, the returned tensor is power spectrogram, which is a real-valued tensor.
                    return_complex = False
                    )   
        
        num_channels, num_frames = signal.shape
        time_axis = torch.arange(0, num_frames) / target_sample_rate
    
        spectrogram = spectrogram_transform(signal)
        
        spectrogram = torch.squeeze(spectrogram)
        spectrogram_db = amplitude_to_db_transform(spectrogram)
    
        n_fft_spec = (spectrogram_db.shape[0] - 1) * 2
        frequency = (target_sample_rate / n_fft_spec) * np.linspace(0, n_fft_spec/2, spectrogram_db.shape[0])
    
        ax = fig.add_subplot(n_rows, n_cols, index+1)
        ax.imshow(spectrogram_db, extent=[0, target_length, 0, target_sample_rate/2], origin="lower", aspect="auto")
        ax.set_title(f"n_fft = {n_fft_value}", fontweight="bold", fontsize=12)
        ax.set_ylabel("Frequency (Hz)")
        ax.set_xlabel("Time (s)")
        
        
fig.suptitle(f"Spectrogram - {event_name} - Class : {event_class}", fontsize=20, fontweight="bold")
plt.tight_layout()
figure_name = f"Analyse des Features - Spectrogramme - Sensibilité au Paramètre n_fft - {event_name} - {event_class}.png"
plt.savefig(os.path.join(figures_path, figure_name), bbox_inches="tight")
plt.show()

### Hop Length

In [ ]:
amplitude_to_db_transform = torchaudio.transforms.AmplitudeToDB(
                    # Scale of input tensor ("power" or "magnitude"). The power being the elementwise square of the magnitude. (Default: "power")
                    stype = "power",
                    # Minimum negative cut-off in decibels. A reasonable number is 80. (Default: None)
                    top_db = None
                    )

hop_length_denominators = [8, 4, 2, 1]

n_cols = 2
n_rows = math.ceil(len(hop_length_denominators)/n_cols)

fig = plt.figure(figsize=(10,10))

for index, hop_length_denominator in enumerate(hop_length_denominators):
    
        spectrogram_transform = torchaudio.transforms.Spectrogram( 
                    # Size of FFT, creates n_fft // 2 + 1 bins. (Default: 400)
                    n_fft = 1024,
                    # Window size. (Default: n_fft)
                    win_length = 1024,
                    # Length of hop between STFT windows. (Default: win_length // 2)
                    hop_length = 1024 // hop_length_denominator,
                    # Two sided padding of signal. (Default: 0)
                    pad = 0,
                    # A function to create a window tensor that is applied/multiplied to each frame/window. (Default: torch.hann_window)
                    window_fn = torch.hann_window,
                    # Exponent for the magnitude spectrogram, (must be > 0) e.g., 1 for energy, 2 for power, etc. (Default: 2)
                    power = 2,
                    # Whether to normalize by magnitude after stft. (Default: False)
                    normalized = True,
                    # Arguments for window function. (Default: None)
                    wkwargs = None,
                    # Whether to pad waveform on both sides so that the t-th frame is centered at time t x hop_length (Default: True)
                    center = False,
                    # Controls the padding method used when center is True. (Default: "reflect")
                    pad_mode = "reflect",
                    # Controls whether to return half of results to avoid redundancy. (Default: True)
                    onesided = True,
                    # Indicates whether the resulting complex-valued Tensor should be represented with native complex dtype, 
                    # such as torch.cfloat and torch.cdouble, or real dtype mimicking complex value with an extra dimension 
                    # for real and imaginary parts. (See also torch.view_as_real.)
                    # This argument is only effective when power=None. It is ignored for cases where power is a number as in those cases, the returned tensor is power spectrogram, which is a real-valued tensor.
                    return_complex = False
                    )   
        
        num_channels, num_frames = signal.shape
        time_axis = torch.arange(0, num_frames) / target_sample_rate
    
        spectrogram = spectrogram_transform(signal)
        spectrogram = torch.squeeze(spectrogram)
        spectrogram_db = amplitude_to_db_transform(spectrogram)
    
        n_fft_spec = (spectrogram_db.shape[0] - 1) * 2
        frequency = (target_sample_rate / n_fft_spec) * np.linspace(0, n_fft_spec/2, spectrogram_db.shape[0])
    
        ax = fig.add_subplot(n_rows, n_cols, index+1)
        ax.imshow(spectrogram_db, extent=[0, target_length, 0, target_sample_rate/2], origin="lower", aspect="auto")
        ax.set_title(f"hop_length = n_fft/{hop_length_denominator}", fontweight="bold", fontsize=12)
        ax.set_ylabel("Frequency (Hz)")
        ax.set_xlabel("Time (s)")
        

fig.suptitle(f"Spectrogram - {event_name} - Class : {event_class}", fontsize=20, fontweight="bold")
plt.tight_layout()
figure_name = f"Analyse des Features - Spectrogramme - Sensibilité au Paramètre hop_length - {event_name} - {event_class}.png"
plt.savefig(os.path.join(figures_path, figure_name), bbox_inches="tight")
plt.show()

### Windowing Function

In [ ]:
amplitude_to_db_transform = torchaudio.transforms.AmplitudeToDB(
                    # Scale of input tensor ("power" or "magnitude"). The power being the elementwise square of the magnitude. (Default: "power")
                    stype = "power",
                    # Minimum negative cut-off in decibels. A reasonable number is 80. (Default: None)
                    top_db = None
                    )

window_functions = [
    ("Hann", torch.hann_window), 
    ("Hamming", torch.hamming_window), 
    ("Bartlett", torch.bartlett_window), 
    ("Blackman", torch.blackman_window), 
    ("Kaiser", torch.kaiser_window)
]

n_cols = 2
n_rows = math.ceil(len(window_functions)/n_cols)

fig = plt.figure(figsize=(10,10))

for index, (window_name, window_function) in enumerate(window_functions):
    
        spectrogram_transform = torchaudio.transforms.Spectrogram( 
                    # Size of FFT, creates n_fft // 2 + 1 bins. (Default: 400)
                    n_fft = 1024,
                    # Window size. (Default: n_fft)
                    win_length = 1024,
                    # Length of hop between STFT windows. (Default: win_length // 2)
                    hop_length = 1024 // 2,
                    # Two sided padding of signal. (Default: 0)
                    pad = 0,
                    # A function to create a window tensor that is applied/multiplied to each frame/window. (Default: torch.hann_window)
                    window_fn = window_function,
                    # Exponent for the magnitude spectrogram, (must be > 0) e.g., 1 for energy, 2 for power, etc. (Default: 2)
                    power = 2,
                    # Whether to normalize by magnitude after stft. (Default: False)
                    normalized = True,
                    # Arguments for window function. (Default: None)
                    wkwargs = None,
                    # Whether to pad waveform on both sides so that the t-th frame is centered at time t x hop_length (Default: True)
                    center = False,
                    # Controls the padding method used when center is True. (Default: "reflect")
                    pad_mode = "reflect",
                    # Controls whether to return half of results to avoid redundancy. (Default: True)
                    onesided = True,
                    # Indicates whether the resulting complex-valued Tensor should be represented with native complex dtype, 
                    # such as torch.cfloat and torch.cdouble, or real dtype mimicking complex value with an extra dimension 
                    # for real and imaginary parts. (See also torch.view_as_real.)
                    # This argument is only effective when power=None. It is ignored for cases where power is a number as in those cases, the returned tensor is power spectrogram, which is a real-valued tensor.
                    return_complex = False
                    )   
        
        num_channels, num_frames = signal.shape
        time_axis = torch.arange(0, num_frames) / target_sample_rate
    
        spectrogram = spectrogram_transform(signal)
        spectrogram = torch.squeeze(spectrogram)
        spectrogram_db = amplitude_to_db_transform(spectrogram)
    
        n_fft_spec = (spectrogram_db.shape[0] - 1) * 2
        frequency = (target_sample_rate / n_fft_spec) * np.linspace(0, n_fft_spec/2, spectrogram_db.shape[0])
    
        ax = fig.add_subplot(n_rows, n_cols, index+1)
        ax.imshow(spectrogram_db, extent=[0, target_length, 0, target_sample_rate/2], origin="lower", aspect="auto")
        ax.set_title(f"window_fn : {window_name}", fontweight="bold", fontsize=12)
        ax.set_ylabel("Frequency (Hz)")
        ax.set_xlabel("Time (s)")
        
        
fig.suptitle(f"Spectrogram - {event_name} - Class : {event_class}", fontsize=20, fontweight="bold")
plt.tight_layout()
figure_name = f"Analyse des Features - Spectrogramme - Sensibilité au Paramètre window_fn - {event_name} - {event_class}.png"
plt.savefig(os.path.join(figures_path, figure_name), bbox_inches="tight")
plt.show()

## Mel-Spectrogram

### Number of Mel Filterbanks

In [ ]:
amplitude_to_db_transform = torchaudio.transforms.AmplitudeToDB(
                    # Scale of input tensor ("power" or "magnitude"). The power being the elementwise square of the magnitude. (Default: "power")
                    stype = "power",
                    # Minimum negative cut-off in decibels. A reasonable number is 80. (Default: None)
                    top_db = None
                    )

n_mels_values = [16, 32, 64, 128]

n_cols = 2
n_rows = math.ceil(len(n_fft_values)/n_cols)

fig = plt.figure(figsize=(10,10))

for index, n_mels_value in enumerate(n_mels_values):
    
        mel_spectrogram_transform = torchaudio.transforms.MelSpectrogram( 
                    # Sample rate of audio signal. (Default: 16000)
                    sample_rate = target_sample_rate,
                    # Size of FFT, creates n_fft // 2 + 1 bins. (Default: 400)
                    n_fft = n_fft_value,
                    # Window size. (Default: n_fft)
                    win_length = n_fft_value,
                    # Length of hop between STFT windows. (Default: win_length // 2)
                    hop_length = n_fft_value // 2,
                    # Minimum frequency. (Default: 0.)
                    f_min = 0,
                    # Maximum frequency. (Default: None)
                    f_max = None,
                    # Two sided padding of signal. (Default: 0)
                    pad = 0,
                    # Number of mel filterbanks. (Default: 128)
                    n_mels = n_mels_value,
                    # A function to create a window tensor that is applied/multiplied to each frame/window. (Default: torch.hann_window)
                    window_fn = torch.hann_window,
                    # Exponent for the magnitude spectrogram, (must be > 0) e.g., 1 for energy, 2 for power, etc. (Default: 2)
                    power = 2,
                    # Whether to normalize by magnitude after stft. (Default: False)
                    normalized = True,
                    # Arguments for window function. (Default: None)
                    wkwargs = None,
                    # whether to pad waveform on both sides so that the tt-th frame is centered at time t \times \text{hop\_length}t×hop_length. (Default: True)
                    center = True,
                    # Controls the padding method used when center is True. (Default: "reflect")
                    pad_mode = "reflect",
                    # Controls whether to return half of results to avoid redundancy. (Default: True)
                    onesided = True,
                    # If ‘slaney’, divide the triangular mel weights by the width of the mel band (area normalization). (Default: None)
                    norm = None,
                    # Scale to use: htk or slaney. (Default: "htk")
                    mel_scale = "htk"
        )   
        
        num_channels, num_frames = signal.shape
        time_axis = torch.arange(0, num_frames) / target_sample_rate
    
        mel_spectrogram = mel_spectrogram_transform(signal)
        mel_spectrogram = torch.squeeze(mel_spectrogram)
        mel_spectrogram_db = amplitude_to_db_transform(mel_spectrogram)
    
        ax = fig.add_subplot(n_rows, n_cols, index+1)
        ax.imshow(mel_spectrogram_db, extent=[0, target_length, 0, n_mels_value], origin="lower", aspect="auto")
        ax.set_title(f"n_mels = {n_mels_value}", fontweight="bold", fontsize=12)
        ax.set_ylabel("Mel Bands")
        ax.set_xlabel("Time (s)")
        
        
fig.suptitle(f"Mel-Spectrogram - {event_name} - Class : {event_class}", fontsize=20, fontweight="bold")
plt.tight_layout()
figure_name = f"Analyse des Features - Mel-Spectrogramme - Sensibilité au Paramètre n_mels - {event_name} - {event_class}.png"
plt.savefig(os.path.join(figures_path, figure_name), bbox_inches="tight")
plt.show()

## MFCC

### Number of Coefficients

In [ ]:
n_mfcc_values = [20, 40, 60, 80]

n_cols = 2
n_rows = math.ceil(len(n_fft_values)/n_cols)

fig = plt.figure(figsize=(10,10))

for index, n_mfcc_value in enumerate(n_mfcc_values):
    
        mfcc_transform = torchaudio.transforms.MFCC( 
                    # Sample rate of audio signal. (Default: 16000)
                    sample_rate = target_sample_rate,
                    # Number of mfc coefficients to retain. (Default: 40)
                    n_mfcc = n_mfcc_value,
                    # Type of DCT (discrete cosine transform) to use. (Default: 2)
                    dct_type = 2,
                    # Norm to use. (Default: "ortho")
                    norm = "ortho",
                    # Whether to use log-mel spectrograms instead of db-scaled. (Default: False)
                    log_mels = False,
                    # Arguments for MelSpectrogram. (Default: None)
                    melkwargs = None
        )   
        
        num_channels, num_frames = signal.shape
        time_axis = torch.arange(0, num_frames) / target_sample_rate
    
        mfcc = mfcc_transform(signal)
        mfcc = torch.squeeze(mfcc)
    
        ax = fig.add_subplot(n_rows, n_cols, index+1)
        ax.imshow(mfcc, extent=[0, target_length, 0, n_mfcc_value], origin="lower", aspect="auto")
        ax.set_title(f"n_mfcc = {n_mfcc_value}", fontweight="bold", fontsize=12)
        ax.set_ylabel("MFCC")
        ax.set_xlabel("Time (s)")
        ax.yaxis.set_major_formatter(FormatStrFormatter('%.0f'))
        
        
fig.suptitle(f"MFCC - {event_name} - Class : {event_class}", fontsize=20, fontweight="bold")
plt.tight_layout()
figure_name = f"Analyse des Features - MFCC - Sensibilité au Paramètre n_mfcc - {event_name} - {event_class}.png"
plt.savefig(os.path.join(figures_path, figure_name), bbox_inches="tight")
plt.show()